# Part 1: DATA MODELING

Implement an algorithm that reads a csv data dump from Reddit and creates a database (relational or non-relational), taking into account the different entities and relationships holding between them. With this database in place, implement queries for generating reports about the dataset.

### Starter code for loading the csv file and connecting to database

In [41]:
import numpy as np
import pandas as pd

data_path = 'data_reddit.csv'
df_reddit = pd.read_csv(data_path,converters={'selftext' : str})    # Put converters here because column of selftext has value of null. but pandas will convert it to NaN

Connect to the MySQL server

In [42]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost:8000',
                             user='user',
                             password='host',
                             database='database',
                             charset='utf8mb4')

---

### SQL and Python code for creating the tables

In [43]:
with connection.cursor() as cur:
    q = """
            DROP TABLE IF EXISTS post,subreddit,user;
    """
    cur.execute(q)
    connection.commit()

with connection.cursor() as cur:
    q = """
        CREATE TABLE subreddit(
        subreddit_id int NOT NULL AUTO_INCREMENT,
        subreddit_name varchar(21) UNIQUE,
        subr_created_at datetime,
        subr_description varchar(500),
        subr_faved_by text,
        subr_numb_members int,
        subr_numb_posts bigint,
        CONSTRAINT subreddit_PK PRIMARY KEY (subreddit_id)
        )DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
    """
    cur.execute(q)
    connection.commit()

with connection.cursor() as cur:
    q = """
        CREATE TABLE user(
        user_id int NOT NULL AUTO_INCREMENT,
        user_name varchar(25) UNIQUE,
        user_num_posts int,
        user_registered_at datetime,
        user_upvote_ratio float(10,9),           
        CONSTRAINT user_PK PRIMARY KEY (user_id)
        )DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
    """
    cur.execute(q)
    connection.commit()

with connection.cursor() as cur:
    q = """
        CREATE TABLE post(
        post_id int NOT NULL AUTO_INCREMENT,
        user_name varchar(25),
        posted_at datetime,
        num_comments int,
        score int,
        selftext text,
        subreddit_name varchar(21),
        title varchar(400),
        total_awards_received int,
        upvote_ratio float(3,2),           
        CONSTRAINT post_PK PRIMARY KEY (post_id),
        CONSTRAINT FK_subreddit FOREIGN KEY (subreddit_name) REFERENCES subreddit(subreddit_name),
        CONSTRAINT FK_user FOREIGN KEY (user_name) REFERENCES user(user_name)
        )DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;
    """
    cur.execute(q)
    connection.commit()

### Python logic for reading in the data

In [44]:
df_tmp=df_reddit[['user_registered_at','user_upvote_ratio']].dropna()   # get the last two colums

df_reddit.drop(labels=['user_registered_at','user_upvote_ratio'],axis=1,inplace=True)   # remove last two colums
df_reddit.dropna(thresh=5,inplace=True)    # delete row if less than 5 non-empty value in this row.
df_reddit.reset_index(drop=True, inplace=True)  # reset the index after droping many rows

df_reddit=pd.concat([df_reddit,df_tmp],axis=1)  # splice two dataframe 

df_reddit['subr_created_at']=pd.to_datetime(df_reddit['subr_created_at'],format='%d/%m/%Y')     # format these columns to datetime which is valid in database
df_reddit['user_registered_at']=pd.to_datetime(df_reddit['user_registered_at'],format='%d/%m/%Y') 
df_reddit['posted_at']=pd.to_datetime(df_reddit['posted_at']) 

df_reddit.fillna(0.0,inplace=True)     # fill the rest of na with 0.0 such as ratio

# filter and drop duplicate rows then assign to 3 dataframe which maps 3 entities
df_user=df_reddit[['author','user_num_posts','user_registered_at','user_upvote_ratio']].drop_duplicates(subset=['author'],keep='last')
df_post=df_reddit[['author','posted_at','num_comments','score','selftext','subreddit','title','total_awards_received','upvote_ratio']].drop_duplicates()
df_subreddit=df_reddit[['subreddit','subr_created_at','subr_description','subr_faved_by','subr_numb_members','subr_numb_posts']].drop_duplicates(subset=['subreddit'])

# Change dataframe to list for inserting database. Actually Pandas can directly use .to_sql()
subreddit_list=df_subreddit.values.tolist()
user_list=df_user.values.tolist()
post_list=df_post.values.tolist()

### SQL and Python code for populating the tables

In [45]:
try:   
    with connection.cursor() as cur:
        q = """
                INSERT INTO user(user_name,user_num_posts,user_registered_at,user_upvote_ratio) VALUES (%s, %s,%s, %s)
        """
        cur.executemany(q, user_list)
        connection.commit()

    with connection.cursor() as cur:
        q = """
                INSERT INTO subreddit(subreddit_name,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts) VALUES (%s, %s, %s, %s, %s, %s)
        """
        cur.executemany(q, subreddit_list)
        connection.commit()

    with connection.cursor() as cur:
        q = """
                INSERT INTO post(user_name,posted_at,num_comments,score,selftext,subreddit_name,title,total_awards_received,upvote_ratio) VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s)
        """
        cur.executemany(q, post_list)
        connection.commit()
finally:
    connection.close()